In [1]:
#  Imports
import pandas as pd
import os

# 경로 설정
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..')) 
data_dir = os.path.join(base_dir, 'data')  # data 폴더 경로
processed_dir = os.path.join(data_dir, 'processed')  # processed 폴더 경로

# 파일 경로 설정
climate_data_path = os.path.join(processed_dir, 'preprocessed_climate.csv')
market_data_path = os.path.join(processed_dir, 'preprocessed_agromarket.csv')

# 데이터 불러오기
dfc = pd.read_csv(climate_data_path)
dfm = pd.read_csv(market_data_path)

In [2]:
#  날짜 컬럼 변환
dfc['일시'] = pd.to_datetime(dfc['일시'], format='%Y-%m-%d')
dfm['일자'] = pd.to_datetime(dfm['일자'].astype(str), format='%Y%m%d')

# 2024-12-20 이후의 데이터 필터링
dfm = dfm[dfm['일자'] < '2024-12-20']

In [3]:
#  서울/수도권 도매시장 필터링
# 서울 및 수도권 도매시장 필터링
capital_region_markets = ['구리', '서울가락', '서울강서', '수원', '인천남촌', '인천삼산', '안산', '안양']
dfm = dfm[dfm['도매시장'].isin(capital_region_markets)]

# 서울 지역 날씨 필터링
dfc = dfc[dfc['지점명'] == '서울']

In [4]:
# 품목별 단위가격 계산 및 일자별 집계
dfm['단위가격(원/kg)'] = dfm['금액(원)'] / dfm['물량(kg)']

# 품목별 일자별 총물량과 평균단가 계산
dfm_daily = (
    dfm.groupby(['품목', '일자'])
    .agg(총물량=('물량(kg)', 'sum'), 평균단가=('단위가격(원/kg)', 'mean'))
    .reset_index()
)


In [5]:
# 평균 현지기압 칼럼 삭제
if '평균 현지기압(hPa)' in dfc.columns:
    dfc = dfc.drop(columns=['평균 현지기압(hPa)'])

In [6]:
# 결측값(NaN) 처리
weather_cols = [
    '평균기온(°C)', '최저기온(°C)', '최고기온(°C)',
    '일강수량(mm)', '최대 풍속(m/s)', '평균 풍속(m/s)',
    '평균 상대습도(%)', '합계 일조시간(hr)', '일 최심신적설(cm)'
]
for col in weather_cols:
    if col in dfc.columns:
        dfc[col] = dfc[col].fillna(0)

In [7]:
# 데이터 결합
dfc = dfc.rename(columns={'일시': '일자'})
df_final = pd.merge(dfm_daily, dfc, on='일자', how='left')

In [8]:
# 파생 변수 생성
df_final['온도차(°C)'] = (df_final['최고기온(°C)'] - df_final['최저기온(°C)']).round(1) 
df_final['월'] = df_final['일자'].dt.month
df_final['계절'] = df_final['월'].apply(lambda x: (x % 12 + 3) // 3)

In [9]:
# 데이터 저장
final_data_path = os.path.join(processed_dir, 'daily_average_price_volume_by_product_seoul.csv')
df_final.to_csv(final_data_path, index=False, encoding='utf-8-sig')

In [10]:
#  기본 정보 확인
print("=== 최종 df_final 기본 정보 ===")
print(df_final.info())
print(df_final.head(10))

=== 최종 df_final 기본 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53244 entries, 0 to 53243
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   품목           53244 non-null  object        
 1   일자           53244 non-null  datetime64[ns]
 2   총물량          53244 non-null  float64       
 3   평균단가         53244 non-null  float64       
 4   지점명          53244 non-null  object        
 5   평균기온(°C)     53244 non-null  float64       
 6   최저기온(°C)     53244 non-null  float64       
 7   최고기온(°C)     53244 non-null  float64       
 8   일강수량(mm)     53244 non-null  float64       
 9   최대 풍속(m/s)   53244 non-null  float64       
 10  평균 풍속(m/s)   53244 non-null  float64       
 11  평균 상대습도(%)   53244 non-null  float64       
 12  합계 일조시간(hr)  53244 non-null  float64       
 13  일 최심신적설(cm)  53244 non-null  float64       
 14  온도차(°C)      53244 non-null  float64       
 15  월            53244 non-null